In [1]:
import torch
import numpy as np
import cv2
import os
from functools import partial
import itertools
import tqdm

import mafat_radar_challenge.data_loader.augmentation as module_aug
import mafat_radar_challenge.data_loader.data_loaders as module_data
import mafat_radar_challenge.model.loss as module_loss
import mafat_radar_challenge.model.metric as module_metric
import mafat_radar_challenge.model.model as module_arch
from mafat_radar_challenge.trainer import Trainer, MAFATTrainer
from mafat_radar_challenge.utils import setup_logger
from mafat_radar_challenge.tester import MAFATTester
import mafat_radar_challenge.data_loader.data_splitter as module_splitter
import mafat_radar_challenge.data_loader.samplers as module_sampler
import mafat_radar_challenge.data_loader.mixers as module_mix

from mafat_radar_challenge.main import get_instance, setup_device, apply_swa
from mafat_radar_challenge.cli import load_config

import matplotlib.pyplot as plt
from sklearn import metrics
from scipy.stats import pearsonr
import pandas as pd

In [2]:
def plot_roc_curve(gt, score, suffix=""):
    fpr, tpr, th = metrics.roc_curve(gt, score)
    auc = round(metrics.auc(fpr, tpr), 4)
    plt.semilogx(fpr, tpr, "-", label="AUC = {}".format(auc) + " " + suffix)
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.grid(which="both")
    plt.legend()
    
def auc_score(gt, preds):
    fpr, tpr, th = metrics.roc_curve(gt, preds)
    return -metrics.auc(fpr, tpr)
    
def precision_recall_curve(gt, score, suffix=""):
    # fig = plt.figure(figsize=(15,10))
    prec, recall, th = metrics.precision_recall_curve(gt, score)
    # auc = metrics.auc(prec, recall)
    plt.plot(prec, recall, "-", label=suffix)#, label="AUC = {}".format(auc))
    plt.xlabel("Precision")
    plt.ylabel("Recall")
    plt.grid(which="both")
    plt.legend()
    
def plot_histogram(positive_preds, negative_preds):
    fig = plt.figure(figsize=(15,10))
    ax1 = fig.add_subplot(111)
    ax2 = ax1.twinx()
    ax2.hist(positive_preds, alpha=0.6, log=False, label='+', cumulative=False, bins=30, color='orange')
    ax1.hist(negative_preds, alpha=0.4, log=False, label='-', cumulative=False, bins=30, color='blue')
    ax1.legend(loc=1)
    ax2.legend(loc=2)
    plt.show()
    
def plot_stacked_histogram(df, grouping_col="snr_type"):
    group_values = df[grouping_col].unique()
    positive_df = df[df.target_type == 1]
    negative_df = df[df.target_type == 0]    
        
    fig = plt.figure(figsize=(15,10))
    ax1 = fig.add_subplot(111)
    ax2 = ax1.twinx()
    pos_colors = [y for x,y in zip(group_values, ["orange", "lime", "yellow"])]
    neg_colors = [y for x,y in zip(group_values, ["blue", "purple", "cyan"])]
    ax2.hist([positive_df.loc[positive_df[grouping_col] == x, "score"].values.tolist() for x in group_values], alpha=0.6, log=False, label=[x + ' +' for x in group_values], stacked=True, cumulative=False, bins=30, color=pos_colors)
    ax1.hist([negative_df.loc[negative_df[grouping_col] == x, "score"].values.tolist() for x in group_values], alpha=0.4, log=False, label=[x + ' -' for x in group_values], stacked=True, cumulative=False, bins=30, color=neg_colors)
    ax1.legend(loc=2)
    ax2.legend(loc=1)
    plt.show()
    
def plot_calibration_curves(gt, score, suffix=""):
    fpr, tpr, th = metrics.roc_curve(gt, score)
    fpr = np.clip(fpr, 0, 1)
    tpr = np.clip(tpr, 0, 1)
    th = np.clip(th, 0, 1)
    fnr = 1 - tpr
    plt.plot(th, fpr, label="FPR" + " " + suffix)
    plt.plot(th, fnr, label="FNR" + " " + suffix)
    #plt.yscale("log")
    plt.xlabel("TH")
    plt.ylabel("FPR & FNR")
    plt.grid(which="both")
    plt.legend()

In [3]:
def predict(model_path, checkpoint=None, data_path=None, metadata_path=None, filter_synth=False, filter_aux=True):
    print(model_path)
    # Setup
    cfg = load_config(os.path.join(os.path.dirname(model_path), "config.yml"))
    transforms = get_instance(module_aug, "augmentation", cfg)
    if "sampler" in cfg:
        sampler = getattr(module_sampler, cfg["sampler"]["type"])
        sampler = partial(sampler, **cfg["sampler"]["args"])
    else:
        sampler = None

    if "mixer" in cfg:
        mixer = get_instance(module_mix, "mixer", cfg)
    else:
        mixer = None
    if (data_path is not None) and (metadata_path is not None):
        cfg["val_data_loader"]["args"]["batch_size"] = 32
        cfg["val_data_loader"]["args"]["data_dir"] = data_path
        cfg["val_data_loader"]["args"]["csv_dir"] = metadata_path
    valid_data_loader = get_instance(module_data, "val_data_loader", cfg, transforms, sampler)
    validation_df = valid_data_loader.dataset.df.copy()
    model = get_instance(module_arch, "arch", cfg)
    if checkpoint is None:
        checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint["state_dict"])
    model, device = setup_device(model, cfg["target_devices"])
    model.eval()

    # Predict
    counter = 0
    preds = list()
    gt = list()
    with torch.no_grad():
        for idx, (image_batch, label_batch) in enumerate(valid_data_loader):
            if isinstance(image_batch, list):
                for i, _ in enumerate(image_batch):
                    image_batch[i] = image_batch[i].to(device)
                data = image_batch
            else:
                data = image_batch.to(device)
            output = model(data).cpu()
            output = torch.sigmoid(output)
            preds.append(output.cpu().numpy())
            for idx_2, (image, label) in enumerate(zip(image_batch, label_batch)):
                gt.append(label[0].cpu().numpy())

    preds = np.vstack(preds).reshape(-1).tolist()
    gt = np.vstack(gt).reshape(-1).tolist()
    
    validation_df["label"] = gt
    validation_df["score"] = preds
    
    if filter_synth:
        validation_df = validation_df[validation_df.source != "synth"]
    if filter_aux:
        aux_segments = validation_df.loc[validation_df.source == "aux", "segment_id"]
        validation_df = validation_df[~validation_df.segment_id.isin(aux_segments)]
        
    return validation_df

In [4]:
# Take best model, predict and calculate score

In [5]:
base_models = [
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v5_adam_centered/0905-131049/checkpoints/checkpoint-epoch13.pth"
]

In [6]:
base_models = [
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v9_adam_fold0/0922-202644/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v9_adam_fold1/0923-175040/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v9_adam_fold2/0923-200711/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v9_adam_fold3/0923-222449/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v9_adam_fold4/0924-004209/checkpoints/model_best.pth",   
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v8_adam_fold0/0924-200924/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v8_adam_fold1/0924-222622/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v8_adam_fold2/0925-004336/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v8_adam_fold3/0925-030037/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v8_adam_fold4/0925-051739/checkpoints/model_best.pth",   
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b3_more_aux_more_synth_specaug_simple_aug_v9_adam_fold0/0925-151207/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b3_more_aux_more_synth_specaug_simple_aug_v9_adam_fold1/0925-185430/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b3_more_aux_more_synth_specaug_simple_aug_v9_adam_fold2/0925-223701/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b3_more_aux_more_synth_specaug_simple_aug_v9_adam_fold3/0926-021930/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b3_more_aux_more_synth_specaug_simple_aug_v9_adam_fold4/0926-060202/checkpoints/model_best.pth",   
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_more_aug_v9_adam_centered0/0927-185118/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_more_aug_v9_adam_centered1/0927-210936/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_more_aug_v9_adam_centered2/0927-232928/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_more_aug_v9_adam_centered3/0928-014604/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_more_aug_v9_adam_centered4/0928-040237/checkpoints/model_best.pth",   
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_less_aug_v9_adam_centered0/0928-210739/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_less_aug_v9_adam_centered1/0928-232816/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_less_aug_v9_adam_centered2/0929-014620/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_less_aug_v9_adam_centered3/0929-040430/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_less_aug_v9_adam_centered4/0929-062240/checkpoints/model_best.pth",   
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_less_aug_v9_adam_centered_blend0/0929-225220/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_less_aug_v9_adam_centered_blend1/0930-011037/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_less_aug_v9_adam_centered_blend2/0930-032751/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_less_aug_v9_adam_centered_blend3/0930-054506/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_less_aug_v9_adam_centered_blend4/0930-080225/checkpoints/model_best.pth",   
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b1_more_aux_more_synth_specaug_simple_aug_v9_adam_centered0/0930-213909/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b1_more_aux_more_synth_specaug_simple_aug_v9_adam_centered1/0930-234106/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b1_more_aux_more_synth_specaug_simple_aug_v9_adam_centered2/1001-014329/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b1_more_aux_more_synth_specaug_simple_aug_v9_adam_centered3/1001-034550/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b1_more_aux_more_synth_specaug_simple_aug_v9_adam_centered4/1001-054814/checkpoints/model_best.pth",   
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v9_adam_pw_0.9_fold0/1001-195710/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v9_adam_pw_0.9_fold1/1001-221433/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v9_adam_pw_0.9_fold2/1002-003156/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v9_adam_pw_0.9_fold3/1002-024950/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v9_adam_pw_0.9_fold4/1002-050747/checkpoints/model_best.pth",   
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v9_adam_focal_fold0/1002-072547/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v9_adam_focal_fold1/1002-094356/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v9_adam_focal_fold2/1002-120142/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v9_adam_focal_fold3/1002-141913/checkpoints/model_best.pth",
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_more_aux_more_synth_specaug_simple_aug_v9_adam_focal_fold4/1002-163722/checkpoints/model_best.pth",   
#     "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b4_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold0/1003-101910/checkpoints/model_best.pth",
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b4_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold1/1003-175906/checkpoints/model_best.pth",
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b4_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold2/1004-013832/checkpoints/model_best.pth",
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b4_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold3/1004-091304/checkpoints/model_best.pth",
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b4_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold4/1004-164923/checkpoints/model_best.pth",   
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_pruned_more_aux_more_synth_specaug_simple_aug_v9_adam_centered0/1005-005432/checkpoints/model_best.pth",
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_pruned_more_aux_more_synth_specaug_simple_aug_v9_adam_centered1/1005-021455/checkpoints/model_best.pth",
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_pruned_more_aux_more_synth_specaug_simple_aug_v9_adam_centered2/1005-033519/checkpoints/model_best.pth",
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_pruned_more_aux_more_synth_specaug_simple_aug_v9_adam_centered3/1005-045546/checkpoints/model_best.pth",
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b2_pruned_more_aux_more_synth_specaug_simple_aug_v9_adam_centered4/1005-061620/checkpoints/model_best.pth",   
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b5_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold0/1005-184411/checkpoints/model_best.pth",
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b5_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold1/1006-082322/checkpoints/model_best.pth",
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b5_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold2/1006-220329/checkpoints/model_best.pth",
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b5_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold3/1007-114115/checkpoints/model_best.pth",
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b5_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold4/1008-012514/checkpoints/model_best.pth",   

]

In [7]:
# Take averaged model and see if it performs the same after the bn_update
epoch_range = [2, 3]
score_dict = {}
for model in tqdm.tqdm(base_models):
    # df = predict(model, data_path="/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_full_public_test_v7_spectrogram.npy", metadata_path="/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_full_public_test_set.csv")
    df = predict(model)
    best_score = auc_score(df.label, df.score)
    print("Base score: ",  best_score)
    best_checkpoint = {}
    for epochs in epoch_range:
        swa_model, ensemble_path = apply_swa(model, epochs, save=False)
        # df = predict(model, checkpoint={"state_dict": swa_model.state_dict()}, data_path="/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_full_public_test_v7_spectrogram.npy", metadata_path="/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_full_public_test_set.csv")
        df = predict(model, checkpoint={"state_dict": swa_model.state_dict()})
        new_score = auc_score(df.label, df.score)
        print("New score: ",  new_score)
        if new_score < best_score:
            print("Improved score")
            best_score = new_score
            best_checkpoint = {"state_dict": swa_model.state_dict()}
            best_ensemble_path = ensemble_path
    if best_checkpoint:
        print(best_ensemble_path, best_score)
        score_dict[best_ensemble_path] = best_score
        torch.save(best_checkpoint, ensemble_path)
        torch.save(best_checkpoint, os.path.join(os.path.dirname(ensemble_path), "model_best-swa.pth"))

  0%|          | 0/14 [00:00<?, ?it/s]

/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b4_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold1/1003-175906/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b4
Base score:  -0.9827229299363057
Loaded pretrained weights for efficientnet-b4
Start epoch:  11
Finished
Start epoch:  12
Finished
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b4_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold1/1003-175906/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b4
New score:  -0.9789129252757496
Loaded pretrained weights for efficientnet-b4
Start epoch:  11
Finished
Start epoch:  12
Finished
Start epoch:  13
Finished
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b4_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold1/1003-175906/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b4


  7%|▋         | 1/14 [15:35<3:22:43, 935.69s/it]

New score:  -0.9839890476930246
Improved score
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b4_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold1/1003-175906/checkpoints/checkpoint-epoch11-13.pth -0.9839890476930246
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b4_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold2/1004-013832/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b4
Base score:  -0.9785179825101614
Loaded pretrained weights for efficientnet-b4
Start epoch:  15
Finished
Start epoch:  16
Finished
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b4_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold2/1004-013832/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b4
New score:  -0.9665691587633944
Loaded pretrained weights for efficientnet-b4
Start epoch:  15
Finished
Start epoch:  16
Finished
Start epoch:  17
Finished
/mnt/agarcia_HD

 14%|█▍        | 2/14 [31:09<3:07:02, 935.24s/it]

...

New score:  -0.9730416307427023
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b5_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold3/1007-114115/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b5
Base score:  -0.9846278260748321
Loaded pretrained weights for efficientnet-b5
Start epoch:  12
Finished
Start epoch:  13
Finished
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b5_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold3/1007-114115/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b5
New score:  -0.9851297746111641
Improved score
Loaded pretrained weights for efficientnet-b5
Start epoch:  12
Finished
Start epoch:  13
Finished
Start epoch:  14
Finished
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b5_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold3/1007-114115/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b5


 93%|█████████▎| 13/14 [3:22:22<24:33, 1473.05s/it]

/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b5_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold4/1008-012514/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b5
Base score:  -0.9928073737397041
Loaded pretrained weights for efficientnet-b5
Start epoch:  14
Finished
Start epoch:  15
Finished
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b5_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold4/1008-012514/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b5
New score:  -0.9886390728222166
Loaded pretrained weights for efficientnet-b5
Start epoch:  14
Finished
Start epoch:  15
Finished
Start epoch:  16
Finished
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_replica_aug_eff_b5_more_aux_more_synth_specaug_simple_aug_v9_adam_centered_fold4/1008-012514/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b5


100%|██████████| 14/14 [3:52:36<00:00, 996.87s/it] 

New score:  -0.991503818378977


In [15]:
auc_score(df.label, df.score)

-0.8944808417708953

In [12]:
auc_score(df.label, df.score)

-0.9454536430414929

- Models trained with full data

In [4]:
base_models = [
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_b2_all_data/1012-234226/checkpoints/model_best.pth",
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_b3_all_data/1013-022349/checkpoints/model_best.pth",
    "/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_b4_all_data/1013-064711/checkpoints/model_best.pth"
]

In [5]:
# Take averaged model and see if it performs the same after the bn_update
epoch_range = [2, 3, 4]
score_dict = {}
for model in tqdm.tqdm(base_models):
    # df = predict(model, data_path="/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_full_public_test_v7_spectrogram.npy", metadata_path="/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_full_public_test_set.csv")
    df = predict(model)
    best_score = auc_score(df.label, df.score)
    print("Base score: ",  best_score)
    best_checkpoint = {}
    for epochs in epoch_range:
        swa_model, ensemble_path = apply_swa(model, epochs, save=False)
        # df = predict(model, checkpoint={"state_dict": swa_model.state_dict()}, data_path="/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_full_public_test_v7_spectrogram.npy", metadata_path="/home/agarcia/repos/mafat-radar-challenge/mafat_radar_challenge/data/mafat_full_public_test_set.csv")
        df = predict(model, checkpoint={"state_dict": swa_model.state_dict()})
        new_score = auc_score(df.label, df.score)
        print("New score: ",  new_score)
        if new_score < best_score:
            print("Improved score")
            best_score = new_score
            best_checkpoint = {"state_dict": swa_model.state_dict()}
            best_ensemble_path = ensemble_path
    if best_checkpoint:
        print(best_ensemble_path, best_score)
        score_dict[best_ensemble_path] = best_score
        torch.save(best_checkpoint, ensemble_path)
        torch.save(best_checkpoint, os.path.join(os.path.dirname(ensemble_path), "model_best-swa.pth"))

  0%|          | 0/3 [00:00<?, ?it/s]

/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_b2_all_data/1012-234226/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b2
Base score:  -0.9620806035338495
Loaded pretrained weights for efficientnet-b2
Start epoch:  5
Finished
Start epoch:  6
Finished
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_b2_all_data/1012-234226/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b2
New score:  -0.931159420289855
Loaded pretrained weights for efficientnet-b2
Start epoch:  5
Finished
Start epoch:  6
Finished
Start epoch:  7
Finished
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_b2_all_data/1012-234226/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b2
New score:  -0.9179571173317451
Loaded pretrained weights for efficientnet-b2
Start epoch:  5
Finished
Start epoch:  6
Finished
Start epoch:  7
Finished
Start epoch:  8
Finished
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_b2_all_data/1012-234226/ch

 33%|███▎      | 1/3 [06:39<13:18, 399.07s/it]

New score:  -0.9238137780424855
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_b3_all_data/1013-022349/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b3
Base score:  -0.9240619416319239
Loaded pretrained weights for efficientnet-b3
Start epoch:  10
Finished
Start epoch:  11
Finished
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_b3_all_data/1013-022349/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b3
New score:  -0.9114552312884654
Loaded pretrained weights for efficientnet-b3
Start epoch:  10
Finished
Start epoch:  11
Finished
Start epoch:  12
Finished
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_b3_all_data/1013-022349/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b3
New score:  -0.9261465157832043
Improved score
Loaded pretrained weights for efficientnet-b3
Start epoch:  10
Finished
Start epoch:  11
Finished
Start epoch:  12
Finished
Start epoch:  13
Finished
/mnt/agarcia_HDD/mafat-ra

 67%|██████▋   | 2/3 [17:31<07:55, 475.10s/it]

New score:  -0.9119515584673417
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_b3_all_data/1013-022349/checkpoints/checkpoint-epoch10-12.pth -0.9261465157832043
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_b4_all_data/1013-064711/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b4
Base score:  -0.9285288862418105
Loaded pretrained weights for efficientnet-b4
Start epoch:  10
Finished
Start epoch:  11
Finished
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_b4_all_data/1013-064711/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b4
New score:  -0.9446595195552908
Improved score
Loaded pretrained weights for efficientnet-b4
Start epoch:  10
Finished
Start epoch:  11
Finished
Start epoch:  12
Finished
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_b4_all_data/1013-064711/checkpoints/model_best.pth
Loaded pretrained weights for efficientnet-b4
New score:  -0.9375124081794719
Loaded pretrained weights for efficie

100%|██████████| 3/3 [39:15<00:00, 785.10s/it]

New score:  -0.9184038117927338
/mnt/agarcia_HDD/mafat-radar-challenge-experiment/MAFAT_b4_all_data/1013-064711/checkpoints/checkpoint-epoch10-11.pth -0.9446595195552908
